In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob
from sklearn.metrics import classification_report
from sklearn.metrics import plot_roc_curve
import matplotlib.pyplot as plt

#Data preprocessing to removes @usernames,urls,symbols and makes all text lowercase
def preprocess_text(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', text)
    text = re.sub('@[^\s]+','', text)
    text = text.lower().replace("ё", "е")
    text = re.sub('[^a-zA-Zа-яА-Я]+',' ', text)
    text = re.sub(' +',' ', text)
    return text.strip()


#from google.colab import files   
#uploaded = files.upload()
#nltk.download('punkt')

#Global variable
batchsize = 100000 

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load data from csv/excel/json
#data=pd.read_csv('textblob_sample_5k.csv',encoding = 'unicode_escape')
#data=pd.read_csv('textblob_sentiment_1M.csv',encoding = "ISO-8859–1")
data = pd.read_csv('/content/drive/MyDrive/vote_json_all_no_conflict.csv')

data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,created_at,coordinates,place,retweet_count,favorite_count,favorited,lang,u_screen_name,u_id_str,u_location,u_created_at,u_friends_count,u_verified,u_followers_count,Tweet ID,text,SentiStrength,Vader,Textblob,Vote
0,Wed Jul 01 13:32:31 +0000 2020,NaN,NaN,291,237,False,en,readyletsgo27,986716551935016960,"Georgia, USA",Wed Apr 18 21:22:13 +0000 2018,25265,False,28552,1278320583718178816,Business Group Complains Trump H-1B Reform Boo...,Negative,Negative,Negative,Negative
1,Wed Jul 01 15:16:16 +0000 2020,NaN,NaN,831,1721,False,en,amvetsupport,755946560794558464,"Palm Desert, CA",Thu Jul 21 02:04:39 +0000 2016,37780,False,90076,1278346691171586049,That moron trump vows to veto the Defense Bill...,Neutral,Negative,Negative,Negative
2,Wed Jul 01 16:44:49 +0000 2020,NaN,NaN,0,0,False,en,simosez,85319648,"Florida, USA",Mon Oct 26 13:52:24 +0000 2009,91,False,20,1278368975689220097,@JoeBiden Debate President Trump. PROVE you do...,Neutral,Neutral,Neutral,Neutral
3,Wed Jul 01 16:44:49 +0000 2020,NaN,"{'id': '6c686af766d8429c', 'url': 'https://api...",0,4,False,en,stahmosh,19199518,Fort Lauderdale Florida,Mon Jan 19 20:14:13 +0000 2009,365,False,380,1278368976960184320,@PamelaStovall6 @ChuckGrassley @realDonaldTrum...,Neutral,Positive,Neutral,Neutral
4,Wed Jul 01 16:44:48 +0000 2020,NaN,NaN,4,95,False,en,bigbrain4me,1162151724183932928,"Philadelphia, PA",Thu Aug 15 23:59:05 +0000 2019,155,False,105,1278368971314597890,@Jorgensen4POTUS @RealSpikeCohen Just found ou...,Positive,Positive,Positive,Positive


In [ ]:
#data["processed_text"] =  data['full_text'].apply(preprocess_text)
data["processed_text"] = data['text'].apply(preprocess_text)

In [ ]:
pd.options.display.max_colwidth = 150
data.head()


,created_at,coordinates,place,retweet_count,favorite_count,favorited,lang,u_screen_name,u_id_str,u_location,u_created_at,u_friends_count,u_verified,u_followers_count,Tweet ID,text,SentiStrength,Vader,Textblob,Vote,processed_text
0,Wed Jul 01 13:32:31 +0000 2020,NaN,NaN,291,237,False,en,readyletsgo27,986716551935016960,"Georgia, USA",Wed Apr 18 21:22:13 +0000 2018,25265,False,28552,1278320583718178816,Business Group Complains Trump H-1B Reform Boosting U.S. Graduates. Big tech is whining that they will have to hire American instead of cheap fore...,Negative,Negative,Negative,Negative,business group complains trump h b reform boosting u s graduates big tech is whining that they will have to hire american instead of cheap foreign...
1,Wed Jul 01 15:16:16 +0000 2020,NaN,NaN,831,1721,False,en,amvetsupport,755946560794558464,"Palm Desert, CA",Thu Jul 21 02:04:39 +0000 2016,37780,False,90076,1278346691171586049,"That moron trump vows to veto the Defense Bill if it includes renaming bases. So once again, military salaries and defense preparedness are second...",Neutral,Negative,Negative,Negative,that moron trump vows to veto the defense bill if it includes renaming bases so once again military salaries and defense preparedness are second i...
2,Wed Jul 01 16:44:49 +0000 2020,NaN,NaN,0,0,False,en,simosez,85319648,"Florida, USA",Mon Oct 26 13:52:24 +0000 2009,91,False,20,1278368975689220097,@JoeBiden Debate President Trump. PROVE you don’t have dementia. #DementiaJoeCantDebate #JoeBidenScaredToDebate,Neutral,Neutral,Neutral,Neutral,debate president trump prove you don t have dementia dementiajoecantdebate joebidenscaredtodebate
3,Wed Jul 01 16:44:49 +0000 2020,NaN,"{'id': '6c686af766d8429c', 'url': 'https://api.twitter.com/1.1/geo/id/6c686af766d8429c.json', 'place_type': 'city', 'name': 'Fort Lauderdale', 'fu...",0,4,False,en,stahmosh,19199518,Fort Lauderdale Florida,Mon Jan 19 20:14:13 +0000 2009,365,False,380,1278368976960184320,@PamelaStovall6 @ChuckGrassley @realDonaldTrump Democrats are always saving @GOP dinosaurs like grassley after they fvck up,Neutral,Positive,Neutral,Neutral,democrats are always saving dinosaurs like grassley after they fvck up
4,Wed Jul 01 16:44:48 +0000 2020,NaN,NaN,4,95,False,en,bigbrain4me,1162151724183932928,"Philadelphia, PA",Thu Aug 15 23:59:05 +0000 2019,155,False,105,1278368971314597890,@Jorgensen4POTUS @RealSpikeCohen Just found out about you and so far I love your policies and what you have to say. Ofc I still need to read more ...,Positive,Positive,Positive,Positive,just found out about you and so far i love your policies and what you have to say ofc i still need to read more about your stances and plans but i...


In [ ]:
#Converting the text into tokens and getting the counts of each token based on the ngrams specified. 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = nltk.word_tokenize)
text_counts = cv.fit_transform(data['processed_text'])

In [ ]:
#import nltk
#nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
text_counts

<2475775x247849 sparse matrix of type '<class 'numpy.int64'>'
	with 25453762 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
#X_train, X_test, Y_train, Y_test = train_test_split(text_counts,data['Sentiment'], test_size=0.25, random_state=5)
X_train, X_test, Y_train, Y_test = train_test_split(text_counts,data['Vote'], test_size=0.25, random_state=5)

In [ ]:
def batch_xy(input, output, batchsize):
      for i in range(0, input.shape[0], batchsize):
        yield input[i:i + batchsize, :], output[i:i + batchsize]

def batch_x(input, batchsize):
      for i in range(0, input.shape[0], batchsize):
        yield input[i:i + batchsize, :]

def report(name, target, pred):
    label = "[" + name + "] classification report:"
    print(label)
    print(classification_report(target, pred)) 

def roc(name, model, input, target):
    label = "[" + name + "] roc curve:"
    print(label)
    plt.figure()
    roc_plt = plot_roc_curve(model, input, target)
    plt.show()


In [ ]:
#Linear SVM
from sklearn.linear_model import SGDClassifier
svc=  SGDClassifier()
for x, y in batch_xy(X_train, Y_train, batchsize):
      svc.partial_fit(x, y, classes=['Positive','Neutral','Negative'])

y_pred = np.array([])

for x in batch_x(X_test, batchsize):
      y_pred = np.append(y_pred, svc.predict(x))

accuracy_score_mnb = metrics.accuracy_score(y_pred, Y_test)
print('accuracy_score_mnb = '+str('{:4.2f}'.format(accuracy_score_mnb*100))+'%')
report("Initial SVC with text counts from Count Vectorizer", Y_test, y_pred)


accuracy_score_mnb = 86.40%
[Initial SVC with text counts from Count Vectorizer] classification report:
              precision    recall  f1-score   support

    Negative       0.90      0.83      0.87    192007
     Neutral       0.83      0.94      0.88    261925
    Positive       0.89      0.79      0.83    165012

    accuracy                           0.86    618944
   macro avg       0.87      0.85      0.86    618944
weighted avg       0.87      0.86      0.86    618944

